#### Names of people in the group

Please write the names of the people in your group in the next cell.

Name of person A Vegard Vaeng Bernhardsen

Name of person B None

In [0]:
# We need to install 'ipython_unittest' to run unittests in a Jupyter notebook
!pip install -q ipython_unittest

You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-ac06c9a4-eac3-44e1-940e-3bf0904510fd/bin/python -m pip install --upgrade pip' command.


In [0]:
# Loading modules that we need
import unittest
from pyspark.sql.dataframe import DataFrame
from pyspark.sql import SparkSession
from typing import Any

In [0]:
# A helper function to load a table (stored in Parquet format) from DBFS as a Spark DataFrame 
def load_df(table_name: "name of the table to load") -> DataFrame:
    return spark.read.parquet(table_name)

users_df = load_df("/user/hive/warehouse/users")
comments_df = load_df("/user/hive/warehouse/comments")
posts_df = load_df("/user/hive/warehouse/posts")

#### Subtask 1: implenenting two helper functions
Impelment these two functions:
1. 'run_query' that gets a Spark SQL query and run it on df which is a Spark DataFrame; it returns the content of the first column of the first row of the DataFrame that is the output of the query;
2. 'run_query2' that is similar to 'run_query' but instead of one DataFrame gets two; it returns the content of the first column of the first row of the DataFrame that is the output of the query.

Note that the result of a Spark SQL query is itself a Spark DataFrame.

In [0]:
def run_query(query: "a SQL query string", df: "the DataFrame that the query will be executed on") -> Any:
    # Get the current Spark session
    spark = SparkSession.builder.getOrCreate()
    
    # Register the DataFrame as a temporary view
    df.createOrReplaceTempView("tempView")
    
    # Execute the query
    result_df = spark.sql(query)
    
    # Fetch the content of the first column of the first row
    result = result_df.collect()[0][0]
    
    return result

def run_query2(query: "a SQL query string", df1: "DataFrame A", df2: "DataFrame B") -> Any:
   # Get the current Spark session
    spark = SparkSession.builder.getOrCreate()
    
    # Register both DataFrames as temporary views
    df1.createOrReplaceTempView("tempView1")
    df2.createOrReplaceTempView("tempView2")
    
    # Execute the query
    result_df = spark.sql(query)
    
    # Fetch the content of the first column of the first row
    result = result_df.collect()[0][0]
    
    return result

In [0]:
# Loading 'ipython_unittest' so we can use '%%unittest_main' magic command
%load_ext ipython_unittest

The ipython_unittest extension is already loaded. To reload it, use:
  %reload_ext ipython_unittest


In [0]:
posts_df.createOrReplaceTempView("posts")
users_df.createOrReplaceTempView("users")
comments_df.createOrReplaceTempView("comments")


#### Subtask 2: writing a few queries
Write the following queries in SQL to be executed by Spark in the next cell.

1. 'q1': find the 'Id' of the most recently created post ('df' is 'posts_df') 
2. 'q2': find the number users
3. 'q3': find the 'Id' of the user who posted most number of answers
4. 'q4': find the number of questions
5. 'q5': find the display name of the user who posted most number of comments

Note that 'q1' is already available below as an example. Moreover, remmebr that Spark supports ANSI SQL 2003 so your queries have to comply with that standard.

In [0]:
q1 = "SELECT * FROM posts ORDER BY CreationDate DESC limit 1"

q2 = "SELECT COUNT(*) AS NumberOfUsers FROM users"

q3 = "SELECT OwnerUserId AS UserId, COUNT(*) AS NumberOfAnswers FROM posts WHERE PostTypeId = 2 GROUP BY OwnerUserId ORDER BY NumberOfAnswers DESC LIMIT 1"

q4 = "SELECT COUNT(*) AS NumberOfQuestions FROM posts WHERE PostTypeId = 1"

# For q5, assuming you need to join users and comments, adjust the query to fit your data structure and temporary view names
q5 = "SELECT DisplayName FROM users JOIN comments ON users.Id = comments.UserId GROUP BY DisplayName ORDER BY COUNT(*) DESC LIMIT 1"



#### Subtask 3: validating the implementations by running the tests

Run the cell below and make sure that all the tests run successfully.

In [0]:
%%unittest_main
class TestTask2(unittest.TestCase):
    def test_q1(self):
        # find the id of the most recent post
        r = run_query(q1, posts_df)
        self.assertEqual(r, 95045)

    def test_q2(self):
        # find the number of the users
        r = run_query(q2, users_df)
        self.assertEqual(r, 91616)

    def test_q3(self):
        # find the user id of the user who posted most number of answers
        r = run_query(q3, posts_df)
        self.assertEqual(r, 64377)

    def test_q4(self):
        # find the number of questions
        r = run_query(q4, posts_df)
        self.assertEqual(r, 28950)

    def test_q5(self):
        # find the display name of the user who posted most number of comments
        r = run_query2(q5, users_df, comments_df)
        self.assertEqual(r, "Neil Slater")

Success

.....
----------------------------------------------------------------------
Ran 5 tests in 7.775s

OK
Out[118]: <unittest.runner.TextTestResult run=5 errors=0 failures=0>

#### Subtask 4: answering to questions about Spark related concepts

Please answer the following questions. Write your answer in one to two short paragraphs. Don't copy-paste; instead, write your own understanding.

1. What is the difference between 'DataFrame', 'Dataset', and 'Resilient Distributed Datasets (RDD)'? 
2. When do you suggest using RDDs instead of using DataFrames?
3. What is the main benefit of using DataSets instead of DataFrames?

Write your answers in the next cell.

Your answers...

1:
RDD is the foundational data structure in Spark. it represents an immutable, distributed collection of objects that can be processed in parallel. For processing tasks, RDDs provide low level functionality and fine grained control over data processing tasks, which allows users to apply custom transformations and actions. They are resilient against failures, as they leverage lineage information to compute lost data. 

Dataframe is a distributed collection of data, which is organized into named columns. Which is similar to a table in a relational database. DataFrames are in fact built on top of RDDs and provide a higher level abstrction, which allows users to use SQL like operations to manipulate data. By using Spark's Catalyst optimizer to optimize, it can lead to more efficient execution plans. 

Datasets are a type-safe version of DataFrames which provides the benefits of RDDs with the optimizations of DataFrames. Datasets are available in statically typed languages as Java and Scala. This allows for compile-time type checking and more efficient de/serialization. They offer a blend of RDD's functional programming capabilities and DataFrame's optimization stratergies. 

2:
RDDs are better suited for low level transformations and actions requiring fine-grained control over the data processing tasks. Especially when you're working with unstructured data (text files) or while performing comples operations that are not easy to express in SQL. Furthermore, when you need to maintain precise control over physical data distribution and partitioning across clusters to optimize the performance for some types of computations.

3:
The main benefit for using Datasets over DataFrames is the type safety provided. As DataFrames offer a dynamic schema and allow for easy and epressive manipulations of structured data, they do not however provide compile-time type checking. Which might lead to runtime errors if the ata is accessed using the incorrect column names or data types. 

Datasets however, offer "the ebst of both worlds". They provide the same level of expressiveness as DataFrames, while still providing type safety. Which allows for errors to be caught at compile time in statically typed languages. Making Datasets useful for applications requiring the robustness of type checking together with the performance optimizations of Spark SQL's engine. 